# 📘 02_model_definition.ipynb
## Capítulo 3: Modelo cosmológico y ruido autoconsistente

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# --- 1. Parámetros del modelo ---
params = {
    'm_phi': 1.0,
    'm_chi': 1.0,
    'lambda_phi': 0.1,
    'lambda_chi': 0.1,
    'g': 0.2,
    'V0': 0.0,
    'alpha_phi': 1.0,
    'alpha_chi': 1.0,
    'tau_phi': 2.0,
    'tau_chi': 2.0
}

In [ ]:
# --- 2. Potencial V(phi, chi) ---
def V(phi, chi, p=params):
    V_phi = -0.5 * p['m_phi']**2 * phi**2 + (p['lambda_phi']/4) * phi**4
    V_chi = 0.5 * p['m_chi']**2 * chi**2 + (p['lambda_chi']/4) * chi**4
    V_int = 0.5 * p['g']**2 * phi**2 * chi**2 + p['V0']
    return V_phi + V_chi + V_int

In [ ]:
# --- 3. Ecuación de Friedmann ---
def Hubble(phi, chi, dphi, dchi, p=params):
    rho = 0.5 * (dphi**2 + dchi**2) + V(phi, chi, p)
    return np.sqrt(rho)

In [ ]:
# --- 4. Temperatura de Gibbons–Hawking ---
def T_GH(H):
    return H / (2 * np.pi)

In [ ]:
# --- 5. Ruido OU autoconsistente ---
def OU_noise(zeta, tau, Gamma, Tgh, dt):
    decay = -zeta / tau
    diffusion = np.sqrt(2 * Gamma * Tgh / tau**2) * np.random.normal(0, np.sqrt(dt))
    return decay + diffusion

In [ ]:
# --- 6. Derivadas del potencial ---
def dV_dphi(phi, chi, p):
    return -p['m_phi']**2 * phi + p['lambda_phi'] * phi**3 + p['g']**2 * phi * chi**2

def dV_dchi(phi, chi, p):
    return p['m_chi']**2 * chi + p['lambda_chi'] * chi**3 + p['g']**2 * chi * phi**2

In [ ]:
# --- 7. Ecuaciones de movimiento ---
def dynamics(t, Y, p=params):
    phi, dphi, chi, dchi, zeta_phi, zeta_chi = Y
    H = Hubble(phi, chi, dphi, dchi, p)
    Tgh = T_GH(H)
    Gamma_phi = p['alpha_phi'] * 3 * H
    Gamma_chi = p['alpha_chi'] * 3 * H

    ddphi = -3 * H * dphi - dV_dphi(phi, chi, p) + zeta_phi
    ddchi = -3 * H * dchi - dV_dchi(phi, chi, p) + zeta_chi

    dzeta_phi = OU_noise(zeta_phi, p['tau_phi'], Gamma_phi, Tgh, dt=0.01)
    dzeta_chi = OU_noise(zeta_chi, p['tau_chi'], Gamma_chi, Tgh, dt=0.01)

    return [dphi, ddphi, dchi, ddchi, dzeta_phi, dzeta_chi]

In [ ]:
# --- 8. Condiciones iniciales ---
Y0 = [0.1, 0.0, 0.1, 0.0, 0.0, 0.0]
t_span = (0, 50)
t_eval = np.linspace(*t_span, 5000)

In [ ]:
# --- 9. Integración numérica ---
sol = solve_ivp(lambda t, y: dynamics(t, y, params), t_span, Y0, t_eval=t_eval, method='RK45')

In [ ]:
# --- 10. Visualización básica ---
plt.figure(figsize=(10, 4))
plt.plot(sol.t, sol.y[0], label='ϕ(t)')
plt.plot(sol.t, sol.y[2], label='χ(t)')
plt.xlabel('Tiempo')
plt.ylabel('Campos')
plt.title('Evolución de ϕ y χ con ruido OU autoconsistente')
plt.legend()
plt.grid()
plt.show()